In [ ]:
from stable_baselines3.common.base_class import BaseAlgorithm

from app.configuration.config import TinvestSettings, PgSqlSettings
from datetime import datetime, timezone, timedelta
from gymnasium import Env

from app.models.data.candle import Interval
from app.models.feature_composer import FeatureComposer
from app.models.gym_env_factory import GymEnvFactory, register_single_asset_trading_env
from app.repositories.tinvest_candles import TinvestCandlesRepository
from app.services.tinvest import TinvestAdapter

In [ ]:
tinvest_config = TinvestSettings(_env_file='../app/.env')
tinvest = TinvestAdapter(tinvest_config)

sql_config = PgSqlSettings(_env_file='../app/.env')
candles_repo = TinvestCandlesRepository(sql_config)

register_single_asset_trading_env()

In [ ]:
#symbol = tinvest.get_instrument(ticker="SBER", class_code="TQBR")

#end_date_utc = datetime.now(timezone.utc)
#days_count = 10
#start_date_utc = end_date_utc - timedelta(days=days_count)

#candles = tinvest.get_candles(
#    instrument_id=symbol.uid,
#    start_date_utc=start_date_utc,
#    end_date_utc=end_date_utc,
#    interval=Interval.CANDLE_INTERVAL_10_MIN)

In [ ]:
#candles_repo.insert_candles("SBER", CandleInterval.CANDLE_INTERVAL_10_MIN, candles)

In [ ]:
#df = candles_repo.get_candles("SBER", Interval.CANDLE_INTERVAL_10_MIN.value)
df = candles_repo.get_last_candles("SBER", Interval.CANDLE_INTERVAL_10_MIN.value)
fk = FeatureComposer(fill_missing_values=True)
df = fk.compose(df)
# df

In [ ]:
env_factory = GymEnvFactory(df)
env, total_steps = env_factory.create_env()
#observation, info = env.reset()
#observation, info

In [ ]:
# TODO: обучить и сохранить модель в БД
# TODO: загрузить модель из БД


In [ ]:
# TODO: Move it to predictor
def predict(env: Env, model: BaseAlgorithm) -> dict:
    observation, info = env.reset()
    is_done = False
    while not is_done:
        action, _ = model.predict(observation=observation, deterministic=True)
        observation, reward, terminated, truncated, info = env.step(action)
        is_done = terminated or truncated

    # выход из цикла вернул данные по последней свече
    # используем их для реального предсказания
    action, _ = model.predict(observation=observation, deterministic=True)
    info.update({'action': action})
    return info
